### Desafio: Suicidios (Probabilidade Condicional)

---
**Descrição do Problema:** Dado que você é uma investigadora da polícia e tem em mãos uma base de dados sobre o histórico de suícidios de seu país. 

<ol>
    <li> Crie uma função que ao informar o sexo e a faixa etária de um suícidio é retornado as probilidades do método utilizado.
    <li> Com auxílio da função input() faça a chamada da função do item 1. (a ideia é simular uma interface amigável)
</ol>

**Código do dataset:** Suicide

**Dicas:**

<ul>
    <li> Utilizar o método groupby
</ul>

---

**Passo 1:** Importar as bibliotecas

In [1]:
from pydataset import data
import pandas as pd

**Passo 2:** Importar o dataset e verificar sua estrutura

In [2]:
df = data('Suicide')
df.head()

,Freq,sex,method,age,age.group,method2
1,4,male,poison,10,10-20,poison
2,0,male,cookgas,10,10-20,gas
3,0,male,toxicgas,10,10-20,gas
4,247,male,hang,10,10-20,hang
5,1,male,drown,10,10-20,drown


**Passo 3:** Elimina colunas com informações que não serão utilizadas 

In [3]:
df = df.drop(['age', 'method'], axis=1)
df.head()

,Freq,sex,age.group,method2
1,4,male,10-20,poison
2,0,male,10-20,gas
3,0,male,10-20,gas
4,247,male,10-20,hang
5,1,male,10-20,drown


**Passo 4:** Traduz as informações

In [4]:
# dict com a tradução dos valores da coluna method2
translated_methods = {
    'drown' : 'afogamento',
    'gas' : 'gas',
    'gun' : 'arma de fogo',
    'hang' : 'enforcamento',
    'jump' : 'queda livre',
    'knife' : 'faca',
    'other' : 'outro', 
    'poison' : 'veneno',   
}

# dict com a tradução da coluna sex
translated_sex = {
    'female' : 'feminino',
    'male' : 'masculino'
}

# renomeia as colunas
df.columns = ['Freq', 'Sexo', 'FaixaEtaria', 'Metodo']
# traduz os valores da coluna Metodo e Sexo
df.replace({'Metodo' : translated_methods, 'Sexo' : translated_sex}, inplace=True)
# visualiza a tabela
df.head()

,Freq,Sexo,FaixaEtaria,Metodo
1,4,masculino,10-20,veneno
2,0,masculino,10-20,gas
3,0,masculino,10-20,gas
4,247,masculino,10-20,enforcamento
5,1,masculino,10-20,afogamento


**Passo 5:** Cria a função descrita no item 1

In [5]:
''' Solução 1
'''
def create_report1(df, sexo, f_etaria):
    # retorna o sub-conjunto de dados com o sexo e faixa etária informado
    new_df = df.query("Sexo == @sexo and FaixaEtaria == @f_etaria")
    # como agora o sexo e faixa etaria tem os mesmos valores, elimina essas colunas 
    new_df = new_df.drop(['Sexo', 'FaixaEtaria'], axis=1)
    # agrupa por método e soma os valores (existe mais de 1 exemplo para cada combinação sexo, metodo e etaria)
    new_df = new_df.groupby(by=['Metodo']).sum()
    # ordena a tabela de acorda com os valores da coluna Freq (do maior valor para o menor valor)
    new_df = new_df.sort_values(by='Freq', ascending=False)
    # cria uma coluna Probabilidade que indica a probabilidade do evento ocorrer dado as informações
    # dos atributos das informações Prob = Freq/(Soma de todas as frequências)
    new_df['Probabilidade'] = new_df['Freq']/new_df['Freq'].sum()
    # retorna o resultado
    return new_df


In [10]:

''' Solução 2
'''
def create_report2(df, sexo, f_etaria):
    # agrupa os resultados (somando as ocorrências iguais) em Sexo, FaixaEtaria e Metodo
    new_df = df.groupby(by=['Sexo', 'FaixaEtaria', 'Metodo']).sum()
    # recupera as informações do Sexo e Faixa Etaria desejada
    new_df = new_df.loc[(sexo, f_etaria)]
    # ordena a tabela de acorda com os valores da coluna Freq (do maior valor para o menor valor)
    new_df = new_df.sort_values(by='Freq', ascending=False)
    # cria uma coluna Probabilidade que indica a probabilidade do evento ocorrer dado as informações
    # dos atributos das informações Prob = Freq/(Soma de todas as frequências)
    new_df['Probabilidade'] = new_df['Freq']/new_df['Freq'].sum()
    # retorna o resultado
    return new_df

**Passo 6:** Testar a função

In [6]:
# para facilitar, verificar os grupos de faixa etária contido no dataset
df['FaixaEtaria'].unique()

array(['10-20', '25-35', '40-50', '55-65', '70-90'], dtype=object)

In [7]:
# testa a função create report 1
r = create_report1(df, 'feminino', '10-20')
# mostra o resultado da coluna Probabilidade em porcentagem
r.style.format({'Probabilidade' : '{:.2%}'})

,Freq,Probabilidade
Metodo,,
veneno,921,62.65%
enforcamento,212,14.42%
queda livre,131,8.91%
outro,100,6.80%
gas,40,2.72%
afogamento,30,2.04%
arma de fogo,25,1.70%
faca,11,0.75%


In [8]:
# testa a função create report 2
r = create_report1(df, 'feminino', '10-20')
# mostra o resultado da coluna Probabilidade em porcentagem
r.style.format({'Probabilidade' : '{:.2%}'})

,Freq,Probabilidade
Metodo,,
veneno,921,62.65%
enforcamento,212,14.42%
queda livre,131,8.91%
outro,100,6.80%
gas,40,2.72%
afogamento,30,2.04%
arma de fogo,25,1.70%
faca,11,0.75%


**Passo 7:** Cria um buscador (Item 2) com uma interface "amigável" para "leigos"

In [11]:
''' Sistema de Informações sobre Suicídio (Interface mais amigável)
'''
sexo = input('Informe o sexo: ')
f_etaria = input('Informe a faixa etária: ')
relatorio = create_report1(df, sexo, f_etaria)
relatorio.style.format({'Probabilidade' : '{:.2%}'})

,Freq,Probabilidade
Metodo,,
